### 복습 
- ratings_train.txt 데이터 로드 
- document 컬럼의 결측치 제외
- 텍스트 정규화(특수문자 제거, 2칸 이상의 공백 제외, 문자열 좌우 공백 제거)
- document의 중복 데이터를 제거 
- 데이터프레임에서 상위 1000개만 사용
- texts -> 데이터프레임의 document 컬럼의 values
- labels -> 데이터프레임의 label 컬럼의 values
- 토큰화 -> Komoran 
    - 선택하는 품사는 NNP NNG VV VA MAG XR
    - 불용어 하다, 되다, 이다
- 단어 사전 생성 (vocab)
    - 단어들 중 최소 출현 횟수가 2회 
    - 단어 사전에는 <PAD>, <UNK>을 제일 앞에 지정하여 단어사전 생성
- 토큰화 된 문서들을 단어 사전의 위치 값에 맞게 인코딩 (enc_inputs)

In [1]:
import pandas as pd 
import re 
from konlpy.tag import Komoran
from collections import Counter
# import collections

In [20]:
df = pd.read_csv("../data/ratings_train.txt", sep = '\t')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        150000 non-null  int64 
 1   document  149995 non-null  object
 2   label     150000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.4+ MB


In [21]:
# document 컬럼의 결측치를 제거하시오
df.dropna(subset="document", inplace=True)

In [22]:
def normalize(text):

    text = re.sub(r"[^가-힣0-9a-zA-Z\s\.]", " ", text)
    text = re.sub(r"\s+", " ", text).strip()

    return text

In [ ]:
# document 만 텍스트 정규화 -> Series 데이터에서 normalize() 함수를 실행
# 여러 개의 컬럼의 텍스트를 정규화 -> DataFrame 데이터에서 normalize() 함수를 실행


# Series 데이터에서 각각의 values들을 normalize() 대입 
df['document'].map(normalize)  # 결과를 어디에 대입? -> df['document']
# DataFrame 데이터에서 각각의 values들을 normalize() 대입 
df[ ['id', 'document'] ] = df[ ['id', 'document'] ].astype(str).applymap(normalize)

In [24]:
df.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화 스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [26]:
# document 컬럼의 중복 데이터를 제거 
df.drop_duplicates(subset='document', inplace=True)

In [28]:
# df = df.head(1000)
df = df[:1000]

In [30]:
# 토큰화 작업 
komoran = Komoran()
allow_pos = ["NNP", "NNG", "VV", "VA", "MAG", "XR"]
stop_word = ['하다', '되다', '이다']

def tokenize(doc):
    tokens = []
    for word, pos in komoran.pos(doc):
        if word not in stop_word and pos in allow_pos:
            tokens.append(word)
    return tokens

In [ ]:
# texts, labels 변수 생성 document, lable 데이터를 대입 
texts, labels = df['document'].values, df['label'].values
# texts들을 이용하여 토큰화 
tokens_list = [ tokenize(doc) for doc in texts ]
tokens_list

In [ ]:
# 최소 등장 횟수 2회인 단어사전을 생성 
# 토큰화 된 tokens_list에서 단어들의 출현 횟수를 확인 
freq = Counter( word for toks in tokens_list for word in toks )
freq

In [ ]:
# tokens_list에 있는 token들을 하나의 변수에 대입 
t_s = []
for toks in tokens_list:
    for word in toks:
        t_s.append(word)
Counter(t_s)

In [ ]:
# 단어사전 생성 
min_count = 2

# 단어사전 처음에는 특수 토큰 2개를 먼저 대입
f_vocab = ['<PAD>', '<UNK>']
b_vocab = []
for word, cnt in freq.items():
    # cnt(출현 횟수)가 min_count(최소 출현 횟수) 보다 크거나 같은 경우 
    if cnt >= min_count:
        b_vocab.append(word)
f_vocab + b_vocab

In [42]:
vocab = ['<PAD>', '<UNK>'] + \
    [ word for word, cnt in freq.items() if cnt >= min_count ]

In [ ]:
# token화 된 문서들을 인코딩 
stoi = {word : idx for idx, word in enumerate(vocab)}
stoi

In [ ]:
stoi2 = dict()
for idx, word in enumerate(vocab):
    stoi2[word] = idx
stoi2

In [46]:
import torch

In [ ]:
# 인코딩 함수 정의 
def encode(toks):
    result = [stoi.get(word, stoi['<UNK>']) for word in toks ]
    return result

enc_inputs = [ torch.tensor(encode(toks) , dtype=torch.long) \
              for toks in tokens_list ]

In [ ]:
enc_inputs

In [49]:
# label 데이터들도 tensor 형태로 변환 
labels_t = torch.tensor(labels, dtype=torch.long)

In [50]:
# 학습 데이터, 검증 데이터 셋트으로 데이터 분할 
from sklearn.model_selection import train_test_split

In [51]:
X_train, X_val, Y_train, Y_val = train_test_split(
    enc_inputs, labels_t, test_size=0.2, random_state=42, 
    stratify= labels_t
)

In [53]:
len(X_train)

800

In [55]:
# 파이토치를 이용해서 1차원 합성곱 학습을 생성 
import torch.nn as nn
# 데이터셋, 데이터로더 
from torch.utils.data import Dataset, DataLoader
# 패딩 토큰 -> 토큰의 길이를 채워주기 위한 기능 
from torch.nn.utils.rnn import pad_sequence

In [56]:
# 딥러닝에서 사용할 데이터를 파이토치에 맞게 변환 
class TextDataset(Dataset):
    def __init__(self, xs, ys):
        # 독립변수를 객체 안에 저장
        self.xs = xs
        # 종속변수를 객체 안에 저장
        self.ys = ys
    def __len__(self):
        return len(self.xs)
    def __getitem__(self, idx):
        return self.xs[idx], self.ys[idx]

In [57]:
# 최대 커널의 개수 -> 몇개까지의 단어들을 묶여서 확인할것인가?
#                   n-gram의 수
MAX_K = 5

# collate_fn -> Dataset에서 배치 만큼 데이터를 가져온 뒤 
# 데이터의 형태를 변경할때 사용하는 함수
def collate_fn(batch):
    xs, ys = zip(*batch)
    pad_id = stoi['<PAD>']  # 0

    # batch -> [ [tensor([]), label], [tensor([]), label] ]

    # 패딩 토큰 채우기 1차 
    # (MAX_K 보다 xs[i] 길이가 작다면 MAX_K 길이로 패딩 토큰을 채워준다)
    fixed = []
    for x in xs:
        if len(x) < MAX_K:
            # 필요한 패딩 토큰 개수 : MAX_K - len(x)
            need = MAX_K - len(x)
            # tensor데이터에 <PAD>를 채워준다.-> 결합 
            x = torch.cat(
                [
                    x,  
                    torch.full( (need, ), pad_id, dtype=torch.long )
                ]
            )
        fixed.append(x)
    # 패딩 토큰 채우기 2차 
    # (xs에서 가장 긴 길이만큼 나머지 xs[i]로 길이를 맞춰서 패딩 토큰 추가)
    xs_pad = pad_sequence(
        fixed, batch_first=True, padding_value= pad_id
    )
    lengths = torch.tensor([len(x) for x in fixed], 
                           dtype=torch.long)
    return xs_pad, torch.stack(ys), lengths

train_loader = DataLoader(
    TextDataset(X_train, Y_train), 
    batch_size= 8, 
    shuffle = True, 
    collate_fn=collate_fn
)
val_loader = DataLoader(
    TextDataset(X_val, Y_val), 
    batch_size=8, 
    shuffle=True, 
    collate_fn= collate_fn
)


In [58]:
import math

In [74]:
# 학습 모델 생성 ( kim CNN )
    # Embedding -> Conv1D(k = 3, 4, 5) -> 
    # max-over-time(해당 구간에서 가장 연관이 높은 구간 선택) -> 
    # concat (kernel별 높은 구간) -> Dropout -> Linear

class TextCNN(nn.Module):
    def __init__(
            self, vocab_size, emb_dim, num_classes, 
            kernel_size = (3, 4, 5), 
            num_channel = 100, 
            pad_idx = 0, 
            dropout = 0.5
    ):
        # vocab_size -> 단어 사전의 길이 
        # emb_dim -> 임베딩 벡터의 차원의 수 
        # num_classes -> 분류 class의 개수
        # kernel_size -> 묶이는 단어의 개수 목록
        # num_channel -> 합성곱 작업 후 출력의 차원의 개수 
        # pad_idx -> 패딩 토큰의 인덱스
        # dropout -> 소실되는 데이터의 비율
        super().__init__()
        self.emb = nn.Embedding(vocab_size, 
                                emb_dim, padding_idx= pad_idx)
        # 1차원 합성곱 모델들
        self.convs = nn.ModuleList(
            [
                nn.Conv1d(in_channels=emb_dim, 
                          out_channels=num_channel, 
                          kernel_size= k) for k in kernel_size
            ]
        )
        # 100차원의 모델이 3개가 생성이 되고 출력들을 단순 결합 
        # 300차원이 되면서 과적합의 위험
        # 일부의 데이터를 소실-> 과적합 방지 
        self.dropout = nn.Dropout(dropout)
        # 선형 모델에서는 분류의 형태로 0, 1의 확률이 출력
        self.fc = nn.Linear(num_channel * len(kernel_size), 
                            num_classes )
        # 초기화
        self._init_weights()
    
    # 벡터 초기화하는 함수 
    def _init_weights(self):
        # 자비에르 초기화
        nn.init.xavier_uniform_(self.emb.weight)
        nn.init.xavier_uniform_(self.fc.weight)
        # 합성곱 모델을 초기화 
        for conv in self.convs:
            # 비선형 구조에서 사용하는 다중 퍼셉트론 Relu()를 이용하는 경우
            # 학습이 안정되도록 사용하는 초기화 
            nn.init.kaiming_uniform_(conv.weight, a = math.sqrt(5))
    
    # 순전파
    def forward(self, x):
        # x -> DataLoader를 통해서 들어오는 입력 데이터 
        # batch가 된 데이터들을 collate_fn에 대입하여 나온 결과를 순전파에 입력
        x = self.emb(x)
        # 배치크기, 시퀀스의 길이 ,임베딩 벡터의 차원의 수 
        x = x.transpose(1, 2)
        # 배치크기, 임베딩 벡터의 차원수, 스퀀스의 길이

        feat_maps = []
        # 다중 구조 -> conv -> relu -> Max
        for conv in self.convs:
            h = torch.relu(conv(x)) # 배치의 크기, relu 차원의 개수, T`: 시퀀스의 길이 - 커널의 사이즈 + 1
            # h중 T`의 최대 값
            h = torch.max( h, dim = 2 ).values  # 배치의 크기, relu 차원의 개수
            feat_maps.append(h)
        # 열을 기준으로 feat_maps 단순 결합 
        z = torch.cat(feat_maps, dim=1)  # 배치의 크기, relu차원의 개수 * len(self.conv)
        # 과적합 방지를 위해 일부 데이터를 0으로 변경 
        z = self.dropout(z)
        # 선형 모델에서 예측 
        logits = self.fc(z)

        return logits

     

In [75]:
# 학습의 루프 생성 

# 모델 생성 
model = TextCNN(
    vocab_size= len(vocab), 
    emb_dim= 128, # 차원의 개수(열의 수)
    num_classes = 2, # 분류 class 종류의 수 (0, 1) -> 2
    kernel_size= (3, 4, 5), # 몇개의 단어들을 묶을것인가
    num_channel= 64, # conv1d에서 output의 차원의 개수
    pad_idx= stoi['<PAD>'], # 패딩 토큰의 위치
    dropout= 0.5 # 고차원 데이터에서 과적합 방지용 손실 데이터의 비율
)

In [76]:
# 옵티마이저 설정 
optimizer = torch.optim.Adam(model.parameters(), lr = 3e-3)
# 손실 함수 설정 ( 예측값과 실제값을 차이를 계산하는 객체 )
criterion = nn.CrossEntropyLoss()

In [77]:
# 학습, 예측을 하는 함수 정의 
def run_epoch(loader, train = True):
    # loader -> model에서 사용할 데이터 
    # train -> 학습 모드, 평가 모드 
    if train:
        model.train()
    else:
        model.eval()

    total_loss = 0.0
    corret = 0 
    total = 0

    for x, y, lengths in loader:
        # train 매개변수에 따라 자동 미분을 활성화 할것인가
        with torch.set_grad_enabled(train):
            logits = model(x)
            loss = criterion(logits, y)
            # 학습 모드라면 -> 옵티마이저, 백워드 
            if train:
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
        total_loss += float(loss.item()) * x.size(0)
        # 예측값 -> [ 확률, 확률 ] -> 높은 확률의 위치
        preds = logits.argmax(dim=1)
        corret += int((preds == y).sum().item())
        total = x.size(0)
    mean_loss = total_loss / total
    acc = corret / total
    return mean_loss, acc

    

In [ ]:
for epoch in range(10):
    tr_loss, tr_acc = run_epoch(train_loader, train=True)
    val_loss, val_acc = run_epoch(val_loader, train=False)
    print(f"epoch({epoch}) : train loss({round(tr_loss, 4)}) train acc({round(tr_acc, 4)})")
    print(f"epoch({epoch}) : val loss({round(val_loss, 4)}) val acc({round(val_acc, 4)})")

In [88]:
# test 예측 함수

@torch.no_grad()
def predict(text):
    toks = tokenize(text)
    ids = torch.tensor(encode(toks), dtype=torch.long)
    pad_id = stoi['<PAD>']

    # 모델에서 최대 커널의 크기를 확인 
    max_k = max([m.kernel_size[0] for m in model.convs])

    if len(ids) < max_k:
        need = max_k - len(ids)
        ids  = torch.cat(
            [
                ids, 
                torch.full(
                    (need, ), pad_id, dtype=torch.long
                )
            ]
        )
    x = ids.unsqueeze(0)
    logits = model(x)
    prob = torch.softmax(logits, dim=1).squeeze(0).tolist()
    pred = int(torch.argmax(logits, dim=1).item())
    return prob, pred

p, yhat = predict( "직원의 태도가 별로였고 실망했다" )
print("예측값 : ", yhat)
print("예측 확률 : ", p)

예측값 :  0
예측 확률 :  [1.0, 1.0710101555622131e-10]
